<a href="https://colab.research.google.com/github/aniketrathod763/aniketrathod763/blob/main/Cointab_Data_Analyst_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import pandas as pd

In [69]:
order_report = pd.read_excel('Company X - Order Report.xlsx')

In [70]:
sku_master = pd.read_excel('/content/Company X - SKU Master.xlsx')



In [71]:
pincode_zones = pd.read_excel('/content/Company X - Pincode Zones.xlsx')

In [72]:
courier_rates = pd.read_excel('/content/Courier Company - Rates.xlsx')

In [73]:
courier_invoice = pd.read_excel('/content/Courier Company - Invoice.xlsx')

In [74]:
# Calculate the total weight of each order
order_report_weight = pd.merge(order_report, sku_master, on='SKU', how='left')
order_report_weight['Total Weight (kg)'] = (order_report_weight['Order Qty'] * order_report_weight['Weight (g)']) / 1000
order_total_weight = order_report_weight.groupby('ExternOrderNo')['Total Weight (kg)'].sum().reset_index()

In [75]:
print(pincode_zone.head())
print(courier_invoice.head())
print(order_zones.head())

   Warehouse_Pincode  Customer_Pincode Zone
0             121003            507101    d
1             121003            486886    d
2             121003            532484    d
3             121003            143001    b
4             121003            515591    d
        AWB Code    Order ID  Charged Weight  Warehouse Pincode  \
0  1091117222124  2001806232            1.30             121003   
1  1091117222194  2001806273            1.00             121003   
2  1091117222931  2001806408            2.50             121003   
3  1091117223244  2001806458            1.00             121003   
4  1091117229345  2001807012            0.15             121003   

   Customer Pincode Zone Type of Shipment  Billing Amount (Rs.)  
0            507101    d  Forward charges                 135.0  
1            486886    d  Forward charges                  90.2  
2            532484    d  Forward charges                 224.6  
3            143001    b  Forward charges                  61.3  
4  

In [76]:
# Determine the zone for each order
pincode_zones.rename(columns={'Warehouse Pincode': 'Warehouse_Pincode', 'Customer Pincode': 'Customer_Pincode'}, inplace=True)
courier_invoice.rename(columns={'Warehouse Pincode': 'Warehouse_Pincode', 'Customer Pincode': 'Customer_Pincode'}, inplace=True)
order_zones = pd.merge(order_total_weight, courier_invoice[['Order ID', 'Warehouse_Pincode', 'Customer_Pincode']], left_on='ExternOrderNo', right_on='Order ID', how='left')
order_zones = pd.merge(order_zones, pincode_zones, on=['Warehouse_Pincode', 'Customer_Pincode'], how='left')


In [77]:
# Merge the order zones with the courier rates
order_rates = pd.merge(order_zones, courier_rates, on='Zone', how='left')

In [78]:
print(order_zones.head())
print(courier_rates.head())

   ExternOrderNo  Total Weight (kg)    Order ID  Warehouse_Pincode  \
0     2001806210              0.220  2001806210             121003   
1     2001806226              0.480  2001806226             121003   
2     2001806229              0.500  2001806229             121003   
3     2001806232              1.302  2001806232             121003   
4     2001806233              0.245  2001806233             121003   

   Customer_Pincode Zone  
0            140604    b  
1            723146    d  
2            421204    d  
3            507101    d  
4            263139    b  
  Zone  Weight Slabs  Forward Fixed Charge  \
0    A          0.50                  29.5   
1    B          1.00                  33.0   
2    C          1.25                  40.1   
3    D          1.50                  45.4   
4    E          2.00                  56.6   

   Forward Additional Weight Slab Charge  RTO Fixed Charge  \
0                                   23.6              13.6   
1               

In [79]:
print('Zone' in order_zones.columns)
print('Zone' in courier_rates.columns)

True
True


In [80]:
# Function to calculate the billing amount
def calculate_billing_amount(row):
    fixed_charge = row['Forward Fixed Charge']
    additional_weight = max(row['Total Weight (kg)'] - row['Weight Slabs'], 0)
    additional_charge = additional_weight * row['Forward Additional Weight Slab Charge']
    return fixed_charge + additional_charge

In [81]:
# Calculate the billing amount for each order
order_rates['Calculated Billing Amount'] = order_rates.apply(calculate_billing_amount, axis=1)

In [61]:
# Merge the calculated billing amount with the actual billing amount
final_comparison = pd.merge(order_rates, courier_invoice[['Order ID', 'Billing Amount (Rs.)']], left_on='Order ID', right_on='Order ID', how='left')

In [82]:
# Determine the charge status for each order
final_comparison['Charge Status'] = 'Correctly Charged'
final_comparison.loc[final_comparison['Calculated Billing Amount'] > final_comparison['Billing Amount (Rs.)'], 'Charge Status'] = 'Under Charged'
final_comparison.loc[final_comparison['Calculated Billing Amount'] < final_comparison['Billing Amount (Rs.)'], 'Charge Status'] = 'Over Charged'

In [83]:
# Count the number of orders for each charge status
charge_status_counts = final_comparison['Charge Status'].value_counts().reset_index()
charge_status_counts.columns = ['Charge Status', 'Count']


In [84]:
# Calculate the total amount for each charge status
charge_status_amounts = final_comparison.groupby('Charge Status')['Billing Amount (Rs.)'].sum().reset_index()
charge_status_amounts.columns = ['Charge Status', 'Amount']

In [85]:
# Merge the counts and amounts
final_result = pd.merge(charge_status_counts, charge_status_amounts, on='Charge Status', how='left')


In [86]:
print(final_result)

       Charge Status  Count   Amount
0  Correctly Charged    174  18432.8
